In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import GRU_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = GRU_Models()
teacher_model = load_models.gru_4
teacher_model.load()
student_model = load_models.result_light_gru_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="result",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/gru_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513, 1]) torch.Size([32])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 91.99%, Train loss: 0.0130, Val loss: 0.1439
Epoch: 2/10, Accuracy: 93.60%, Train loss: 0.0057, Val loss: 0.1044
Epoch: 3/10, Accuracy: 94.38%, Train loss: 0.0049, Val loss: 0.0920
Epoch: 4/10, Accuracy: 94.64%, Train loss: 0.0044, Val loss: 0.0839
Epoch: 5/10, Accuracy: 94.72%, Train loss: 0.0042, Val loss: 0.0797
Epoch: 6/10, Accuracy: 94.87%, Train loss: 0.0040, Val loss: 0.0746
Epoch: 7/10, Accuracy: 94.90%, Train loss: 0.0038, Val loss: 0.0748
Epoch: 8/10, Accuracy: 94.81%, Train loss: 0.0038, Val loss: 0.0730
Epoch: 9/10, Accuracy: 95.15%, Train loss: 0.0037, Val loss: 0.0707
Epoch: 10/10, Accuracy: 95.08%, Train loss: 0.0036, Val loss: 0.0697


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.65%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 94.99%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/result_light_gru_1.pth
